# Deploy a Keras trained model using Amazon SageMaker

### Step 1. Set up

After create a new notebook instance, set the kernel to conda_tensorflow_p36.

The get_execution_role function retrieves the AWS Identity and Access Management (IAM) role you created at the time of creating your notebook instance.

In [ ]:
import boto3, re
from sagemaker import get_execution_role

role = get_execution_role()

### Step 2. Load the Keras model using the json and weights file

If you saved your model in the TensorFlow ProtoBuf format, skip to "Step 4. Convert the TensorFlow model to an Amazon SageMaker-readable format.

#### If the model have json configuration file and weights file

In [ ]:
import keras
from keras.models import model_from_json

json_file = open('/home/ec2-user/SageMaker/model.json', 'r')
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)

model.load_weights('/home/ec2-user/SageMaker/model.h5')
print("Loaded model from disk")

#### if model export by model.save (combine config and weights)

In [ ]:
from keras.models import load_model
model = load_model('/home/ec2-user/SageMaker/model.h5')
print("Loaded model from disk")

### Step 3. Export the Keras model to the TensorFlow ProtoBuf format

In [ ]:
from tensorflow.python.saved_model import builder
from tensorflow.python.saved_model.signature_def_utils import predict_signature_def
from tensorflow.python.saved_model import tag_constants

# Note: This directory structure will need to be followed - see notes for the next section
model_version = '1'
export_dir = 'export/Servo/' + model_version

# Build the Protocol Buffer SavedModel at 'export_dir'
builder = builder.SavedModelBuilder(export_dir)

# Create prediction signature to be used by TensorFlow Serving Predict API
signature = predict_signature_def(
    inputs={"inputs": model.input}, outputs={"score": model.output})

from keras import backend as K

with K.get_session() as sess:
    # Save the meta graph and variables
    builder.add_meta_graph_and_variables(
        sess=sess, tags=[tag_constants.SERVING], signature_def_map={"serving_default": signature})
    builder.save()

### Step 4. Convert TensorFlow model to a SageMaker readable format

#### Tar the entire directory and upload to S3

In [ ]:
import tarfile
with tarfile.open('model.tar.gz', mode='w:gz') as archive:
    archive.add('export', recursive=True)
    
import sagemaker

sagemaker_session = sagemaker.Session()
inputs = sagemaker_session.upload_data(path='model.tar.gz', key_prefix='model')

### Step 5. Deploy the trained model

In [ ]:
!touch train.py

from sagemaker.tensorflow.serving import Model
sagemaker_model = Model(model_data = 's3://' + sagemaker_session.default_bucket() + '/model/model.tar.gz',
                        role = role,
                        framework_version='1.15.2',
                        entry_point = 'train.py')

#### Deploy step takes a long time, after that, sagemaker model and endpoint will be created

In [ ]:
%%time
predictor = sagemaker_model.deploy(initial_instance_count=1,
                                   instance_type='ml.m4.xlarge', endpoint_name='endpoint-name')

### Step 6. Invoke the endpoint

#### Invoke the SageMaker endpoint from the notebook

In [ ]:
endpoint_name = predictor.endpoint

import sagemaker
from sagemaker.tensorflow.model import TensorFlowModel
predictor = sagemaker.tensorflow.model.TensorFlowPredictor(endpoint_name, sagemaker_session)

import numpy as np

data = np.random.randn(1, 160, 160)

predictor.predict(data)

#### Invoke the SageMaker endpoint using a boto3 client

In [ ]:
import json
import boto3
import numpy as np
import io
 
client = boto3.client('runtime.sagemaker')

data = np.random.randn(1, 160, 160, 3).tolist()

response = client.invoke_endpoint(EndpointName=endpoint_name, Body=json.dumps(data), ContentType='application/json')

response_body = response['Body'].read()

data = json.loads(response_body)['predictions']

value = np.argmax(data)

print(value)

### Step 7. Clean up

To avoid incurring unnecessary charges, use the AWS Management Console to delete the resources that you created for this exercise:
https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-cleanup.html